# Выбор локации для скважины

**Описание проекта:**

Добывающая компания «ГлавРосГосНефть» хочет решить, где бурить новую скважину.

Для этого она предоставляет пробы нефти в трёх регионах: в каждом 100 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Возможную прибыль и риски необходимо проанализировать техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Условия задачи:** 

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

**Описание данных:**
- Данные геологоразведки трёх регионов находятся в файлах: 
    - /datasets/geo_data_0.csv
    - /datasets/geo_data_1.csv
    - /datasets/geo_data_2.csv
- *id* — уникальный идентификатор скважины;
- *f0*, *f1*, *f2* — три признака точек (неважно, что они означают, но сами признаки значимы);
- *product* — объём запасов в скважине (тыс. баррелей).

**План работы над задачей:**

1) **Загрузка и подготовка данных** — изучим данные на предмет возникновения в них дубликатов, пропусков и ошибок в типах данных.

2) **Обучение моделей** — обучим как модель линейной регрессии, после чего предскажем средние запасы сырья и показатель метрики RMSE для каждого из трёх регионов.

3) **Подготовимся к расчёту прибыли** — рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.

4) **Найдём среднюю прибыль, 95%-ый доверительный интервал и риск убытков** для каждого региона при помощи техники *Bootstrap*.

5) Сделаем **итоговый вывод** о том, какой регион выбрать для разработки скважин. 

## Импорт и настройка библиотек

In [1]:
!pip install scikit-learn==1.1.3 -q

In [2]:
import pandas as pd
import numpy as np
import seaborn as sb
import os

from scipy import stats as st
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler

## Загрузка и подготовка данных

Считаем csv-файлы:

In [3]:
pth1 = 'C:/pr/sat/geo_data_0.csv'
pth2 = 'C:/pr/sat/geo_data_1.csv'
pth3 = 'C:/pr/sat/geo_data_2.csv'
pth4 = '/datasets/geo_data_0.csv'
pth5 = '/datasets/geo_data_1.csv'
pth6 = '/datasets/geo_data_2.csv'

if os.path.exists(pth1):
    df_1 = pd.read_csv(pth1)
elif os.path.exists(pth4):
    df_1 = pd.read_csv(pth4)
else:
    print('Something is wrong')

if os.path.exists(pth2):
    df_2 = pd.read_csv(pth2)
elif os.path.exists(pth5):
    df_2 = pd.read_csv(pth5)
else:
    print('Something is wrong')

if os.path.exists(pth3):
    df_3 = pd.read_csv(pth3)
elif os.path.exists(pth6):
    df_3 = pd.read_csv(pth6)
else:
    print('Something is wrong')
    
df_1 = df_1.set_index('id')
df_2 = df_2.set_index('id')
df_3 = df_3.set_index('id')

Изучим общую информацию о датасетах. Однако, прежде чем мы приступим к их разбору, напишем **вспомогательную функцию**. Так проводить анализ будет быстрее и удобнее:

In [4]:
def describe_data (data):
    print('Первые пять строк таблицы:')
    display(data.head())
    print('\n', 'Последние пять строк таблицы:')
    display(data.tail())
    print('\n', 'Общая информация о датасете:', '\n')
    display(data.info())
    print('\n', 'Описательные статистики датасета:')
    display(data.describe().round(2))
    print('\n', 'Количество дубликатов в данных:')
    display(data.duplicated().sum())

In [5]:
describe_data(df_1) # Изучаем данные о первом регионе

Первые пять строк таблицы:


,f0,f1,f2,product
id,,,,
txEyH,0.705745,-0.497823,1.221170,105.280062
2acmU,1.334711,-0.340164,4.365080,73.037750
409Wp,1.022732,0.151990,1.419926,85.265647
iJLyR,-0.032172,0.139033,2.978566,168.620776
Xdl7t,1.988431,0.155413,4.751769,154.036647



 Последние пять строк таблицы:


,f0,f1,f2,product
id,,,,
DLsed,0.971957,0.370953,6.075346,110.744026
QKivN,1.392429,-0.382606,1.273912,122.346843
3rnvd,1.029585,0.018787,-1.348308,64.375443
7kl59,0.998163,-0.528582,1.583869,74.040764
1CWhH,1.764754,-0.266417,5.722849,149.633246



 Общая информация о датасете: 

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, txEyH to 1CWhH
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


None


 Описательные статистики датасета:


,f0,f1,f2,product
count,100000.00,100000.00,100000.00,100000.00
mean,0.50,0.25,2.50,92.50
std,0.87,0.50,3.25,44.29
min,-1.41,-0.85,-12.09,0.00
25%,-0.07,-0.20,0.29,56.50
50%,0.50,0.25,2.52,91.85
75%,1.07,0.70,4.72,128.56
max,2.36,1.34,16.00,185.36



 Количество дубликатов в данных:


0

In [6]:
describe_data(df_2) # Изучаем данные о втором регионе

Первые пять строк таблицы:


,f0,f1,f2,product
id,,,,
kBEdx,-15.001348,-8.276000,-0.005876,3.179103
62mP7,14.272088,-3.475083,0.999183,26.953261
vyE1P,6.263187,-5.948386,5.001160,134.766305
KcrkZ,-13.081196,-11.506057,4.999415,137.945408
AHL4O,12.702195,-8.147433,5.004363,134.766305



 Последние пять строк таблицы:


,f0,f1,f2,product
id,,,,
QywKC,9.535637,-6.878139,1.998296,53.906522
ptvty,-10.160631,-12.558096,5.005581,137.945408
09gWa,-7.378891,-3.084104,4.998651,137.945408
rqwUm,0.665714,-6.152593,1.000146,30.132364
relB0,-3.426139,-7.794274,-0.003299,3.179103



 Общая информация о датасете: 

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, kBEdx to relB0
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


None


 Описательные статистики датасета:


,f0,f1,f2,product
count,100000.00,100000.00,100000.00,100000.00
mean,1.14,-4.80,2.49,68.83
std,8.97,5.12,1.70,45.94
min,-31.61,-26.36,-0.02,0.00
25%,-6.30,-8.27,1.00,26.95
50%,1.15,-4.81,2.01,57.09
75%,8.62,-1.33,4.00,107.81
max,29.42,18.73,5.02,137.95



 Количество дубликатов в данных:


0

In [7]:
describe_data(df_3) # Изучаем данные о третьем регионе

Первые пять строк таблицы:


,f0,f1,f2,product
id,,,,
fwXo0,-1.146987,0.963328,-0.828965,27.758673
WJtFt,0.262778,0.269839,-2.530187,56.069697
ovLUW,0.194587,0.289035,-5.586433,62.871910
q6cA6,2.236060,-0.553760,0.930038,114.572842
WPMUX,-0.515993,1.716266,5.899011,149.600746



 Последние пять строк таблицы:


,f0,f1,f2,product
id,,,,
4GxBu,-1.777037,1.125220,6.263374,172.327046
YKFjq,-1.261523,-0.894828,2.524545,138.748846
tKPY3,-1.199934,-2.957637,5.219411,157.080080
nmxp2,-2.419896,2.417221,-5.548444,51.795253
V9kWn,-2.551421,-2.025625,6.090891,102.775767



 Общая информация о датасете: 

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, fwXo0 to V9kWn
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


None


 Описательные статистики датасета:


,f0,f1,f2,product
count,100000.00,100000.00,100000.00,100000.00
mean,0.00,-0.00,2.50,95.00
std,1.73,1.73,3.47,44.75
min,-8.76,-7.08,-11.97,0.00
25%,-1.16,-1.17,0.13,59.45
50%,0.01,-0.01,2.48,94.93
75%,1.16,1.16,4.86,130.60
max,7.24,7.84,16.74,190.03



 Количество дубликатов в данных:


0

**Промежуточный вывод:**

Проблем при выгрузке данных не возникло, очевидных дубликатов нет. Данные были предварительно проиндексированы по столбцу *id* и не требуют дополнительной обработки.

## Обучение и проверка модели

На этом этапе обучим модель линейной регрессии и сделаем при помощи неё предсказания на валидационной выборке. Для этого напишем вспомогательную функцию:

In [8]:
RANDOM_STATE = 42
def predict(data):
    model_lr = LinearRegression()
    X = data.drop(['product'], axis=1) 
    y = data['product']
    X_train, X_valid, y_train, y_valid = train_test_split(
        X,
        y,
        random_state=RANDOM_STATE,
        test_size=0.25
    )
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)
    model_lr.fit(X_train_scaled, y_train)
    predictions = model_lr.predict(X_valid_scaled)
    rmse_score = mean_squared_error(y_valid, predictions, squared=False)
    mean_predicted = predictions.mean() 
    return rmse_score, mean_predicted, predictions, y_valid

Для каждого из регионов сохраним предсказания и правильные ответы на валидационной выборке, а также найдём средний запас предсказанного сырья и показатель метрики RMSE:

In [9]:
rmse_1, mean_1, predict_1, y_valid_1 = predict(df_1) # Модель для первого месторождения
print(f'Cредний запас предсказанного сырья: {mean_1.round(2)}, значение метрики RMSE: {rmse_1.round(2)}')

Cредний запас предсказанного сырья: 92.4, значение метрики RMSE: 37.76


In [10]:
rmse_2, mean_2, predict_2, y_valid_2 = predict(df_2) # Модель для второго месторождения
print(f'Cредний запас предсказанного сырья: {mean_2.round(2)}, значение метрики RMSE: {rmse_2.round(2)}')

Cредний запас предсказанного сырья: 68.71, значение метрики RMSE: 0.89


In [11]:
rmse_3, mean_3, predict_3, y_valid_3 = predict(df_3) # Модель для третьего месторождения
print(f'Cредний запас предсказанного сырья: {mean_3.round(2)}, значение метрики RMSE: {rmse_3.round(2)}')

Cредний запас предсказанного сырья: 94.77, значение метрики RMSE: 40.15


**Промежуточный вывод:**

Наибольший запас предсказанного сырья у третьего месторождения (94.8), наименьший — у второго (68.7).

При этом можем отметить, что именно у второго месторождения модель работает точнее всего (значение метрики RMSE там меньше единицы, а чем ближе значение метрики к нулю, тем точнее модель. В то же время у других месторождений значение метрики в районе 40).

## Подготовка к расчёту прибыли

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины:

In [12]:
TOTAL_POINTS = 500 # При разведке региона исследуют 500 точек
POINTS = 200 # Для разработки выбирают 200 лучших
TOTAL_BUDGET = 10 * 10**9 # Бюджет на разработку скважин
PRICE_BARREL = 450000 # Доход с единицы продукта

In [13]:
BUDGET_PER_WELL = TOTAL_BUDGET / POINTS # Сколько выделяется денег на одну скважину
VOLUME = BUDGET_PER_WELL / PRICE_BARREL # Сколько нужно единиц продукта, чтобы окупить бюджет
print("Достаточный объем = ", round(VOLUME, 0), "тыс. баррелей")

Достаточный объем =  111.0 тыс. баррелей


Также напишем вспомогательные функции для расчёта прибыли и проведения процедуры *Bootstrap*:

In [14]:
def revenew(target, predictions, points, total_budget, price_barrel): # Функция для расчёта прибыли
    predictions_sorted = predictions.sort_values(ascending=False)
    selected = target[predictions_sorted.index][:points]
    revenue = selected.sum() * price_barrel - total_budget
    return revenue

def bootstrap(target, probabilities): # Функция для проведения процедуры Bootstrap
    state = np.random.RandomState(12345)
    values=[]
    for i in range(1000):
        target_subsample = target.sample(replace=True,\
            random_state=state, n=TOTAL_POINTS)
        probs_subsample = probabilities[target_subsample.index]
        values.append(revenew(target_subsample, probs_subsample, POINTS, TOTAL_BUDGET, PRICE_BARREL))
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean = values.mean()
    risk = st.percentileofscore(values, 0).round(2)
    return lower, upper, risk

**Промежуточный вывод:**

Необходимое рассчитанное кол-во единиц продукта для безубыточности составило 111 тыс. баррелей. Однако на данный момент самое крупное предсказанное значение среднего запаса сырья составляет 94.8 тыс. (в третьем регионе). В первом и втором регионах аналогичные значения и того меньше. Этого, очевидно, недостаточно для безубыточной разработки, что свидетельствует о невозможности выбора скважин случайным образом (необходимо отобрать наиболее "богатые" скважины). Для проведения такого отбора были прописаны вспомогательные функции для расчёта прибыли и проведения процедуры *Bootstrap*, которыми мы воспользуемся на следующем шаге.

## Расчёт прибыли и рисков 

Найдём среднюю прибыль, 95%-ый доверительный интервал и риск убытков для каждого региона:

In [15]:
revenue_1 = revenew(y_valid_1, pd.Series(predict_1, index=y_valid_1.index), \
                    POINTS, TOTAL_BUDGET, PRICE_BARREL) # Первый регион
lower_1, upper_1, risk_1 = bootstrap(y_valid_1, pd.Series(predict_1, index=y_valid_1.index))
print(f'Прибыль в первом регионе: {(revenue_1 / 10**9).round(2)} млрд')
print(f'95%-ый доверительный интервал для первого региона: ({(lower_1 / 10**9).round(2)} млрд;\
      {(upper_1 / 10**9).round(2)} млрд)')
print(f'Вероятность убытков в первом регионе: {risk_1}%')

Прибыль в первом регионе: 3.36 млрд
95%-ый доверительный интервал для первого региона: (-0.12 млрд;      0.97 млрд)
Вероятность убытков в первом регионе: 6.1%


In [16]:
revenue_2 = revenew(y_valid_2, pd.Series(predict_2, index=y_valid_2.index), \
                    POINTS, TOTAL_BUDGET, PRICE_BARREL) # Второй регион
lower_2, upper_2, risk_2 = bootstrap(y_valid_2, pd.Series(predict_2, index=y_valid_2.index))
print(f'Прибыль во втором регионе: {(revenue_2 / 10**9).round(2)} млрд')
print(f'95%-ый доверительный интервал для второго региона: ({(lower_2 / 10**9).round(2)} млрд;\
      {(upper_2 / 10**9).round(2)} млрд)')
print(f'Вероятность убытков во втором регионе: {risk_2}%')

Прибыль во втором регионе: 2.42 млрд
95%-ый доверительный интервал для второго региона: (0.06 млрд;      0.91 млрд)
Вероятность убытков во втором регионе: 1.1%


In [17]:
revenue_3 = revenew(y_valid_3, pd.Series(predict_3, index=y_valid_3.index), \
                    POINTS, TOTAL_BUDGET, PRICE_BARREL) # Третий регион
lower_3, upper_3, risk_3 = bootstrap(y_valid_3, pd.Series(predict_3, index=y_valid_3.index))
print(f'Прибыль в третьем регионе: {(revenue_3/ 10**9).round(2)} млрд')
print(f'95%-ый доверительный интервал для третьего региона: ({(lower_3 / 10**9).round(2)} млрд;\
      {(upper_3 / 10**9).round(2)} млрд)')
print(f'Вероятность убытков в третьем регионе: {risk_3}%')

Прибыль в третьем регионе: 2.6 млрд
95%-ый доверительный интервал для третьего региона: (-0.15 млрд;      0.95 млрд)
Вероятность убытков в третьем регионе: 7.1%


**Промежуточный вывод:**

По результатам проведения процедуры *Bootstrap* был выявлен единственный регион с вероятностью убытков меньше 2,5% — второй (данный показатель в нём составляет всего 1,1%). Соответственно, именно этот регион можно считать наиболее подходящим для разработки. Однако, рассчитанная прибыль в нём оказалась наименьшей из трёх регионов и составила 2,42 миллиарда рублей.

При этом нельзя не отметить, что наибольшую прибыль продемонстрировал первый регион (она почти в полтора раза больше, чем во втором), поэтому имеет смысл также подробнее изучить месторождения в нём. 

## Итоговый вывод 

По результатам исследования **наиболее целесообразным** для разработки месторождений оказался **второй регион**. Это обосновано **наименьшей вероятностью убытков — 1.1%. Прогнозируемый доход с данного региона составляет 2,42 миллиарда рублей.**

При этом нельзя не отметить, что наибольшую прибыль продемонстрировал первый регион (она почти в полтора раза больше, чем во втором), поэтому имеет смысл также подробнее изучить месторождения в нём. 

**Коротко о промежуточных выводах и итогах данного анализа:**

- Проблем при выгрузке данных не возникло, очевидных дубликатов в них не оказалось. Данные были предварительно проиндексированы по столбцу id и не требовали дополнительной обработки.
- Необходимое рассчитанное кол-во единиц продукта для безубыточности составило 111 тыс. баррелей. Однако самое крупное из предсказанных значений среднего запаса сырья на момент предсказания модели составляло 94.8 тыс. (в третьем регионе). В первом и втором регионах аналогичные значения были и того меньше.
- Подобных объёмов, очевидно, было недостаточно для безубыточной разработки, что свидетельствовало о невозможности выбора скважин случайным образом (необходимо было отобрать наиболее "богатые" скважины). Для проведения такого отбора были прописаны вспомогательные функции для расчёта прибыли и проведения процедуры *Bootstrap*, на основе которых и было решено выбрать наиболее целесообразным **второй регион**.